In [41]:
import pandas as pd
from pandas.io import sql
import sqlite3

In [42]:
conn = sqlite3.connect('ArticlesScraping.db')

sql.read_sql(
"""
select * from ArticleCorpus
""", con=conn).head(5)

,PageLinks,FullLink,Time,Author,Header,Body,CleanedBody
0,/wealth-manager/news/saturday-papers-rbs-to-ch...,http://citywire.co.uk/wealth-manager/news/satu...,2016-08-20 05:17:37,>HimanshuSingh],Saturday Papers: RBS to charge big clients for...,"[<div class=""article-body"" itemprop=""articleBo...",[\nTop stories\n\nFinancial Times: Royal Bank ...
1,/wealth-manager/news/rathbones-adds-to-london-...,http://citywire.co.uk/wealth-manager/news/rath...,2016-08-19 13:42:56,>JoshuaThurston],Rathbones adds to London office with second HS...,"[<div class=""article-body"" itemprop=""articleBo...",[\nRathbone Brothers has hired its second HSBC...
2,/wealth-manager/news/central-banks-pouring-fue...,http://citywire.co.uk/wealth-manager/news/cent...,2016-08-19 12:52:45,>AndrewMilligan],Central banks pouring fuel on dying embers,[],[]
3,/wealth-manager/news/wealth-manager-olympics-t...,http://citywire.co.uk/wealth-manager/news/weal...,2016-08-19 12:27:35,[],Wealth Manager Olympics: the Relay,"[<div class=""article-body jsArticle-body--star...","[\nIf like us, the Wealth Manager Olympics has..."
4,/wealth-manager/news/jupiters-davies-buys-bomb...,http://citywire.co.uk/wealth-manager/news/jupi...,2016-08-19 11:21:00,>JonathanYarker],Jupiter's Davies buys 'bombed out' domestics,"[<div class=""article-body"" itemprop=""articleBo...","[\n\xa0Steve Davies, manager of the\xa0Jupiter..."


In [43]:
#Need to add a line of code here to remove duplicates in table (based on url? Header? time?)

In [44]:
#Need to add a line of code here to clean up the Author column

In [45]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [59]:
def preprocess(s):
    s=s.lower()
    #stem/lemmatize
    #remove
    return s

from sklearn.feature_extraction.text import CountVectorizer
c_vectorizer = CountVectorizer(preprocessor=preprocess,
                               stop_words='english',
                               min_df=.05)

count_matrix = c_vectorizer.fit_transform(
    sql.read_sql("SELECT CleanedBody from ArticleCorpus",con=conn)['CleanedBody']
    )
n_topics = 10
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=10)
lda.fit(count_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=10, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [60]:
data = sql.read_sql("SELECT FullLink, CleanedBody from ArticleCorpus",con=conn)
results = lda.transform(c_vectorizer.fit_transform(data['CleanedBody']))
topic_df = pd.DataFrame(results,columns=["Topic%s" % i for i in range(10)])
topic_df['FullLink'] = data['FullLink']
topic_df.to_sql(name='Article_Topics',
            con=conn,
            if_exists='replace',
            index=False)
topic_df

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,FullLink
0,0.100007,221.099911,0.100002,0.100015,0.100016,0.100012,0.100012,0.100003,0.100002,0.100020,http://citywire.co.uk/wealth-manager/news/satu...
1,0.100002,0.100019,0.100001,53.099932,0.100019,0.100013,0.100003,0.100001,0.100001,0.100009,http://citywire.co.uk/wealth-manager/news/rath...
2,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,http://citywire.co.uk/wealth-manager/news/cent...
3,0.100004,0.100013,0.100002,30.099937,0.100010,0.100009,0.100007,0.100001,0.100001,0.100016,http://citywire.co.uk/wealth-manager/news/weal...
4,0.100002,0.100015,0.100001,0.100010,0.100010,0.100006,119.099940,0.100001,0.100001,0.100015,http://citywire.co.uk/wealth-manager/news/jupi...
5,0.100006,0.100011,0.100001,0.100008,118.099954,0.100006,0.100005,0.100001,0.100001,0.100007,http://citywire.co.uk/wealth-manager/news/towr...
6,0.100004,155.099926,0.100002,0.100007,0.100009,0.100008,0.100016,0.100003,0.100003,0.100023,http://citywire.co.uk/wealth-manager/news/shel...
7,0.100005,0.100021,0.100002,0.100009,0.100010,0.100007,0.100006,0.100002,0.100002,105.099938,http://citywire.co.uk/wealth-manager/news/hedg...
8,0.100002,0.100013,0.100001,0.100013,0.100015,0.100007,0.100015,0.100002,0.100001,119.099930,http://citywire.co.uk/wealth-manager/news/spec...
9,0.100003,0.100019,0.100002,0.100016,53.099921,0.100004,0.100011,0.100003,0.100002,0.100020,http://citywire.co.uk/wealth-manager/news/auro...


In [63]:
n_top_words = 15
import operator
vocabulary = c_vectorizer.get_feature_names()
for i in range(n_topics):
    best_words_indexes = lda.components_[i].argsort()[:-n_top_words - 1:-1]
    best_words = " ".join([vocabulary[i] for i in best_words_indexes])
    print best_words

u2019 course online industry nhe financial investors trading bank people said provide private set information
u2019s u2019 said market nthe million billion u2013 xa0 year uk times brexit months management
u2019 capital public companies manager investment u2019s million equity fund wealth smaller management opportunities toe
investment xa0 wealth china manager companies asset fund returned u2019s director management years trust overall
xa0 advice service trust retirement said fund office brewin dolphin director offer firm financial year
jersey u2019s eu u2019 guernsey said u2018we opportunities brexit office islands uk outside client business
uk davies oil said rate sector companies manager end year things growth banks opportunity really
mining ftse half said 100 million stocks year management rally fund fall helped free billion
u2019s nthe said brexit u2019 times oil april million versus xa31 period billion introduced xa0
xa0 fund management investors income said companies value millio

In [ ]:
#22, 44 topic 10